# Group volumes
Created by Andy Smet, Kristof Egghe


In order to use the grouped volumes we need to be able to go back and forth between the item_id's and there group_id

In [ ]:
# set to the correct paths
allData=True


path="/home/andy/Desktop/AI_Project/"
# path="./drive/MyDrive/AI Project/"

if allData:
  interactions_csv_path = path+"datasets/AllDataPreprocessed_interactions.csv"
else:
  interactions_csv_path = path+"datasets/TestDataPreprocessed_interactions.csv"
# goodreads_path = '/home/andy/Desktop/AI_Project/datasets/Goodreads/'
books_df_g_path=path+"datasets/books_df_grouped.csv"

# Install Packages

In [ ]:
# Install your required packages here
!pip install --user pandas numpy matplotlib sklearn 
import os
import time
import numpy as np
import pandas as pd
import sklearn
import gzip
import json
from tqdm import tqdm
import os
from collections import Counter
from datetime import datetime
import math
tqdm.pandas() #for progres_apply etc.

start = time.time()

You should consider upgrading via the '/home/andy/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Mount google drive
# from google.colab import drive
# drive.mount('/content/drive')



# Create Dicts
groupdic goes from a book_id to the corresponding group_id
reverseGroupdic goes from a group_id to a list of book_ids

In [ ]:
books_df_g=pd.read_csv(books_df_g_path)  
#turn string into list
books_df_g['book_id'] = books_df_g.book_id.apply(lambda x: x[1:-1].split(','))
print(books_df_g.head())
#make a groupdict where an item_id is matched to its corresponding groupid
groupdic=dict()
counter=0
books_df_g["group_id"]=range(0,len(books_df_g))
for i,row in books_df_g.iterrows():
  j=row['book_id']
  # print(j)
  for book in j:
    # print(book)
    groupdic[int(book)]=counter
  counter+=1
print(groupdic)

#make reversegroupdic where a groupid gives back the list of all book_ids belonging to the group
from collections import defaultdict
reverseGroupdic=defaultdict(list)
for key, value in groupdic.items():
    reverseGroupdic[value].append(key)

import pickle
if allData:
  with open(path+"datasets/AllDataGroupdict" + '.pickle', 'wb') as f:
      pickle.dump(groupdic, f, pickle.HIGHEST_PROTOCOL)
  with open(path+"datasets/AllDataReverseGroupdict" + '.pickle', 'wb') as f:
      pickle.dump(reverseGroupdic, f, pickle.HIGHEST_PROTOCOL)
else:
  with open(path+"datasets/TestDataGroupdict" + '.pickle', 'wb') as f:
      pickle.dump(groupdic, f, pickle.HIGHEST_PROTOCOL)
  with open(path+"datasets/TestDataReverseGroupdict" + '.pickle', 'wb') as f:
      pickle.dump(reverseGroupdic, f, pickle.HIGHEST_PROTOCOL)


      book_id                                         title  \
0   [1242262]       " Superman Returns ":  The Visual Guide   
1  [23491474]   "A" come ignoranza: allenamento anti papero   
2  [23491452]  "A" come ignoranza: esplosione di esplosioni   
3  [24542438]         "A" come ignoranza: principesse crude   
4  [23491481]    "A" come ignoranza: spoiler! Sbranzo muore   

                                  authors          publisher num_pages  \
0  [{'author_id': '4953553', 'role': ''}]               {''}        70   
1  [{'author_id': '5559315', 'role': ''}]  {'Panini Comics'}        64   
2  [{'author_id': '5559315', 'role': ''}]  {'Panini Comics'}        64   
3  [{'author_id': '5559315', 'role': ''}]  {'Panini Comics'}        64   
4  [{'author_id': '5559315', 'role': ''}]  {'Panini Comics'}        64   

   publication_year      author_name  group_id  
0            2006.0  Daniel  Wallace         0  
1            2014.0              Daw         1  
2            2014.0          

In [ ]:
interactions_df_processed=pd.read_csv(interactions_csv_path)  
interactions_df_processed['item_id'] = interactions_df_processed['item_id'].astype('int32')
interactions_df_processed["item_id"]=interactions_df_processed["item_id"]
display(interactions_df_processed.head(5))
print(f"number of unique users: {interactions_df_processed['user_id'].nunique()}")
print(f"number of unique items: {interactions_df_processed['item_id'].nunique()}")
print()

#transform bookids to groupids and make them consecutive
interactions_df_processed["item_id"]=interactions_df_processed["item_id"].apply(lambda x: groupdic[x])
cons_group_dic={}
ctr=0
for i in interactions_df_processed['item_id'].unique():
  cons_group_dic[i]=ctr
  ctr+=1
if allData:
  with open(path+"datasets/AllDataConsGroupdict" + '.pickle', 'wb') as f:
      pickle.dump(cons_group_dic, f, pickle.HIGHEST_PROTOCOL)

else:
  with open(path+"datasets/TestDataConsGroupdict" + '.pickle', 'wb') as f:
      pickle.dump(cons_group_dic, f, pickle.HIGHEST_PROTOCOL)



,user_id,item_id,datetime
0,0,271199,2013-06-21 17:23:44
1,0,287380,2013-06-21 17:24:05
2,0,287381,2013-06-21 17:24:31
3,0,287382,2013-06-21 17:25:05
4,0,287388,2013-06-21 17:25:13


number of unique users: 143570
number of unique items: 26431



In [ ]:
# train_test_split=pd.read_csv(path+"datasets/TestDatafold1.csv")  
# display(train_test_split)
# train_test_split['history'] = train_test_split.history.apply(lambda x:[cons_group_dic[groupdic[int(y)]] if y else y for y in x[1:-1].split(',')])
# train_test_split['future'] = train_test_split.future.apply(lambda x:[cons_group_dic[groupdic[int(y)]] if y else y for y in x[1:-1].split(',')])
# display(train_test_split)